In [606]:
import h5py
import os
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

In [607]:
code_path = "../"
code_path

'../'

In [608]:
import sys
sys.path.append(code_path)

from encoding import *
from preprocessing import get_ordered_representations, normalize_train_test, concat_past_features, lanczosinterp2D, delete_block_edges, shuffle_words
from encoding import nested_blocked_cv, ridge_regression_fit_sklearn, ridge_regression_predict_torch
from analysis import pearson_correlation

## 0. load fmri data and annotation

In [609]:
data_path_prefix = "../data/HP_data/fMRI"
HF_home = "/SWS/llms/nobackup/"
results_path_prefix = "/BRAIN/neuroai_project/work/Encoding_models/results"

In [610]:
# raw fmri data for one subject
fmri_data = np.load(f"{data_path_prefix}/data_subject_I.npy", allow_pickle=True)
fmri_data.shape, fmri_data[:10]

((1211, 25263),
 array([[-1.39391795e+00, -4.95610000e-01, -1.76625698e+00, ...,
         -7.66158058e-01, -2.32445619e-01, -1.46871563e-01],
        [ 5.30454149e-01, -2.32485978e-01,  3.52580458e-01, ...,
         -2.31855518e-03, -2.46482123e-01, -4.29284279e-01],
        [ 1.13308842e+00,  3.31803866e-01, -3.14589030e-01, ...,
          8.15993870e-01,  1.31184511e+00,  1.38407545e+00],
        ...,
        [-1.10624234e+00, -1.08674185e+00, -5.44766153e-01, ...,
          6.70158027e-01,  9.81612876e-01,  8.54572439e-01],
        [-1.58241086e+00, -1.14630569e+00, -5.57836731e-01, ...,
         -3.30246991e-01, -4.22094739e-01, -1.26951335e-01],
        [-2.59743061e+00, -1.55403741e+00, -1.93496114e+00, ...,
          1.01716606e+00,  2.10392591e-01,  1.11087041e-01]],
       shape=(10, 25263)))

In [611]:
# timing of each fmri TRs in seconds
fmri_time = np.load(f"{data_path_prefix}/time_fmri.npy", allow_pickle=True)
fmri_time.shape, fmri_time[:10]

((1351,), array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18], dtype=uint16))

In [612]:
# indices of which TRs belong to which run.
fmri_runs = np.load(f"{data_path_prefix}/runs_fmri.npy", allow_pickle=True)
fmri_runs.shape, fmri_runs[:10]

((1351,), array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=uint16))

In [613]:
# list of words shown as stimuli (sequentially on a screen word by word)
stimuli_words = np.load(f"{data_path_prefix}/words_fmri.npy", allow_pickle=True)
stimuli_words.shape, stimuli_words[:10]

((5176,),
 array(['Harry', 'had', 'never', 'believed', 'he', 'would', 'meet', 'a',
        'boy', 'he'], dtype='<U14'))

In [614]:
# timing in seconds of the words
word_times = np.load(f"{data_path_prefix}/time_words_fmri.npy", allow_pickle=True)
word_times.shape, word_times[:10]

((5176,), array([20. , 20.5, 21. , 21.5, 22. , 22.5, 23. , 23.5, 24. , 24.5]))

## 1. prepare the input sequence for the LLM
(e.g. by changing formatting). Here we change "+" to "\n\n" and "@" to nothing (used to highlight italics)

In [615]:
LLM_input_sequence = []
for word in stimuli_words:
    LLM_input_sequence.append(word) if word != "+" and not "@" in word \
    else LLM_input_sequence.append(word.replace("+", "\n\n").replace("@",""))
assert len(word_times.tolist()) == len(LLM_input_sequence), "different length" # make sure we still have the same length as we have word timings.

In [616]:
import random
from typing import List, Tuple

def shuffle_words(words: List[str], percentage: float, seed: int = 42, exclude_tokens: List[str] = ["\n\n"]) -> Tuple[List[str], List[int]]:

    assert 0 <= percentage <= 1, "percentage must be between 0 and 1"

    # Identify positions of words to consider for shuffling
    shuffle_indices = [i for i, w in enumerate(words) if w not in exclude_tokens]
    num_to_shuffle = int(len(shuffle_indices) * percentage)

    if num_to_shuffle == 0:
        return words.copy(), list(range(len(words)))

    # Randomly sample indices to shuffle
    random.seed(seed)
    indices_to_shuffle = random.sample(shuffle_indices, num_to_shuffle)

    # Extract and shuffle the selected words
    words_to_shuffle = [words[i] for i in indices_to_shuffle]
    random.shuffle(words_to_shuffle)

    # Insert shuffled words back
    shuffled_words = words.copy()
    for idx, new_word in zip(indices_to_shuffle, words_to_shuffle):
        shuffled_words[idx] = new_word

    # Build inverse mapping (from new positions back to original indices)
    inverse_indices = list(range(len(words)))
    for i, new_word in zip(indices_to_shuffle, words_to_shuffle):
        original_index = words.index(new_word)  # Caution: If duplicates exist, this gives first match
        inverse_indices[i] = original_index

    return shuffled_words, inverse_indices


In [642]:
import random
from typing import List, Tuple

def shuffle_sentences(
    words: List[str],
    percentage: float,
    seed: int = 42,
    sentence_end_tokens: Tuple[str] = (".", "!", "?")
) -> Tuple[List[str], List[int]]:

    assert 0 <= percentage <= 1, "percentage must be between 0 and 1"

    # Step 1: Split into sentences
    sentences = []
    current_sentence = []
    for w in words:
        current_sentence.append(w)
        if any(w.endswith(punct) for punct in sentence_end_tokens):
            sentences.append(current_sentence)
            current_sentence = []
    if current_sentence:
        sentences.append(current_sentence)

    # Step 2: Decide how many sentences to shuffle
    num_sentences = len(sentences)
    num_to_shuffle = int(num_sentences * percentage)

    if num_to_shuffle == 0:
        return words.copy(), list(range(len(words)))

    # Step 3: Track sentence with index
    indexed_sentences = list(enumerate(sentences))  # (original_index, sentence)

    # Step 4: Select and shuffle a subset
    random.seed(seed)
    shuffle_indices = random.sample(range(num_sentences), num_to_shuffle)

    # Extract and shuffle selected
    selected = [indexed_sentences[i] for i in shuffle_indices]
    random.shuffle(selected)

    # Replace the selected ones
    shuffled_indexed = indexed_sentences.copy()
    for idx, new_sent in zip(shuffle_indices, selected):
        shuffled_indexed[idx] = new_sent  # still (original_idx, sentence)

    # Step 5: Flatten
    # shuffled_indexed contains tuples like (original_sentence_index, sentence).
    # We're flattening each sentence into a list of words (shuffled_words), while keeping track of:
    # orig_sent_idx: which sentence this word originally came from.
    # word_idx: the position of the word within that sentence.
    shuffled_words = []
    word_source_locs = []
    for orig_sent_idx, sent in shuffled_indexed:
        for word_idx, word in enumerate(sent):
            shuffled_words.append(word)
            word_source_locs.append((orig_sent_idx, word_idx))

    # Step 6: Map original (sent_idx, word_idx) to global index
    # Here we build a mapping from each word's original (sentence index, word index) -> its absolute position in the flattened original list.
    sentence_word_to_global_idx = {}
    global_idx = 0
    for sent_idx, sent in enumerate(sentences):
        for word_idx, _ in enumerate(sent):
            sentence_word_to_global_idx[(sent_idx, word_idx)] = global_idx
            global_idx += 1

    # Step 7: Build inverse_indices
    # Now that we have a list of (original_sentence_index, word_index) for each shuffled word, we can recover its original global 
    # position using the mapping from Step 6. This builds the list inverse_indices that allows to undo the shuffle later.
    inverse_indices = [
        sentence_word_to_global_idx.get((s_idx, w_idx), -1)
        for (s_idx, w_idx) in word_source_locs
    ]

    # Diagnostics
    missing = sum(1 for i in inverse_indices if i == -1)
    if missing > 0:
        print(f"Missing {missing} indices out of {len(inverse_indices)}")

    # Reconstruct original
    reconstructed = [None] * len(words)
    for i, orig_i in enumerate(inverse_indices):
        reconstructed[orig_i] = shuffled_words[i]

    assert reconstructed == words, "Inverse mapping failed!"

    return shuffled_words, reconstructed, inverse_indices


In [543]:
shuffled_sequence, inverse = shuffle_words(LLM_input_sequence, percentage=0.3)


In [643]:
shuffled_sequence, reconstructed, inverse = shuffle_sentences(LLM_input_sequence, percentage=1)


In [644]:
reconstructed

[np.str_('Harry'),
 np.str_('had'),
 np.str_('never'),
 np.str_('believed'),
 np.str_('he'),
 np.str_('would'),
 np.str_('meet'),
 np.str_('a'),
 np.str_('boy'),
 np.str_('he'),
 np.str_('hated'),
 np.str_('more'),
 np.str_('than'),
 np.str_('Dudley,'),
 np.str_('but'),
 np.str_('that'),
 np.str_('was'),
 np.str_('before'),
 np.str_('he'),
 np.str_('met'),
 np.str_('Draco'),
 np.str_('Malfoy.'),
 np.str_('Still,'),
 np.str_('first-year'),
 np.str_('Gryffindors'),
 np.str_('only'),
 np.str_('had'),
 np.str_('Potions'),
 np.str_('with'),
 np.str_('the'),
 np.str_('Slytherins,'),
 np.str_('so'),
 np.str_('they'),
 np.str_("didn't"),
 np.str_('have'),
 np.str_('to'),
 np.str_('put'),
 np.str_('up'),
 np.str_('with'),
 np.str_('Malfoy'),
 np.str_('much.'),
 np.str_('Or'),
 np.str_('at'),
 np.str_('least,'),
 np.str_('they'),
 np.str_("didn't"),
 np.str_('until'),
 np.str_('they'),
 np.str_('spotted'),
 np.str_('a'),
 np.str_('notice'),
 np.str_('pinned'),
 np.str_('up'),
 np.str_('in'),
 np

In [645]:
shuffled_sequence

[np.str_('Blood'),
 np.str_('was'),
 np.str_('pounding'),
 np.str_('in'),
 np.str_('his'),
 np.str_('ears.'),
 np.str_("It's"),
 np.str_('really'),
 np.str_('very'),
 np.str_('selfish'),
 np.str_('of'),
 np.str_('you."'),
 '\n\n',
 np.str_('"And'),
 np.str_("it's"),
 np.str_('really'),
 np.str_('none'),
 np.str_('of'),
 np.str_('your'),
 np.str_('business,"'),
 np.str_('said'),
 np.str_('Harry.'),
 '\n\n',
 np.str_('"Madam'),
 np.str_('Hooch'),
 np.str_('told'),
 np.str_('us'),
 np.str_('not'),
 np.str_('to'),
 np.str_('move'),
 np.str_('--'),
 np.str_("you'll"),
 np.str_('get'),
 np.str_('us'),
 np.str_('all'),
 np.str_('into'),
 np.str_('trouble."'),
 '\n\n',
 np.str_('Harry'),
 np.str_('ignored'),
 np.str_('her.'),
 np.str_('The'),
 np.str_('Fat'),
 np.str_('Lady'),
 np.str_('had'),
 np.str_('gone'),
 np.str_('on'),
 np.str_('a'),
 np.str_('nighttime'),
 np.str_('visit'),
 np.str_('and'),
 np.str_('Hermione'),
 np.str_('was'),
 np.str_('locked'),
 np.str_('out'),
 np.str_('of'),
 np

In [621]:
if -1 in inverse:
    num_missing = inverse.count(-1)
    print(f"Missing indices: {num_missing} out of {len(inverse)}")
else:
    print("All inverse indices are valid.")

All inverse indices are valid.


In [646]:
inverse

[1404,
 1405,
 1406,
 1407,
 1408,
 1409,
 3024,
 3025,
 3026,
 3027,
 3028,
 3029,
 3030,
 3031,
 3032,
 3033,
 3034,
 3035,
 3036,
 3037,
 3038,
 3039,
 5175,
 1386,
 1387,
 1388,
 1389,
 1390,
 1391,
 1392,
 1393,
 1394,
 1395,
 1396,
 1397,
 1398,
 1399,
 1400,
 1401,
 1402,
 1403,
 3435,
 3436,
 3437,
 3438,
 3439,
 3440,
 3441,
 3442,
 3443,
 3444,
 3445,
 3446,
 3447,
 3448,
 3449,
 3450,
 3451,
 3343,
 3344,
 3345,
 3346,
 3347,
 3348,
 3349,
 3350,
 3351,
 3352,
 3353,
 3354,
 3355,
 3356,
 3357,
 3358,
 3359,
 3360,
 3361,
 3362,
 3363,
 3364,
 3365,
 3366,
 3367,
 3368,
 3369,
 3370,
 3371,
 3372,
 3373,
 3374,
 3375,
 3376,
 3377,
 3378,
 3379,
 3380,
 3381,
 3382,
 3383,
 3384,
 3385,
 3386,
 3387,
 3388,
 3389,
 3390,
 3391,
 3392,
 3393,
 3394,
 3395,
 3396,
 3397,
 3398,
 3399,
 3400,
 3401,
 3402,
 3403,
 3404,
 3405,
 3406,
 3407,
 3408,
 3409,
 3410,
 3411,
 3412,
 3413,
 5099,
 5100,
 5101,
 5102,
 5103,
 5104,
 5105,
 5106,
 5107,
 5108,
 5109,
 5110,
 5111,
 5112,

In [308]:
import random
import numpy as np
seed_value=42
# Track the positions of special tokens to exclude from shuffling
shuffle_indices = [i for i, w in enumerate(LLM_input_sequence) if w not in ["\n\n"]]
# Extract and shuffle the words to be shuffled
words_to_shuffle = [LLM_input_sequence[i] for i in shuffle_indices]
shuffled_words = words_to_shuffle.copy()
random.seed(seed_value)
np.random.seed(seed_value)
random.shuffle(shuffled_words)

In [248]:
len(shuffle_indices)

4975

In [249]:
# Rebuild the full sequence with shuffled words and preserved special tokens
final_sequence = LLM_input_sequence.copy()
for idx, word_idx in enumerate(shuffle_indices):
    final_sequence[word_idx] = shuffled_words[idx]

In [250]:
len(LLM_input_sequence)

5176

In [251]:
len(final_sequence)

5176

In [252]:
def compute_inverse_indices(original: List[str], shuffled: List[str]) -> List[int]:
    from collections import defaultdict
    used = defaultdict(int)
    index_map = defaultdict(list)
    
    # Map all occurrences of each word to their positions in the original
    for i, word in enumerate(original):
        index_map[word].append(i)

    indices = []
    for word in shuffled:
        idx = index_map[word][used[word]]
        indices.append(idx)
        used[word] += 1
    return indices

In [253]:
inverse_indices = compute_inverse_indices(original=final_sequence, shuffled=LLM_input_sequence)

In [263]:
inverse_indices

[252,
 1538,
 1507,
 3,
 5021,
 23,
 3142,
 218,
 394,
 3431,
 934,
 3815,
 39,
 25,
 666,
 47,
 1495,
 1192,
 44,
 129,
 894,
 357,
 119,
 1344,
 1412,
 3614,
 130,
 349,
 62,
 478,
 82,
 503,
 246,
 1894,
 131,
 768,
 549,
 71,
 242,
 64,
 31,
 32,
 2,
 158,
 76,
 147,
 495,
 33,
 60,
 146,
 70,
 645,
 435,
 2054,
 1434,
 281,
 45,
 104,
 4976,
 153,
 3801,
 517,
 1237,
 1699,
 1743,
 575,
 900,
 1150,
 2692,
 150,
 237,
 113,
 1294,
 497,
 21,
 1621,
 54,
 34,
 368,
 79,
 1827,
 771,
 874,
 1173,
 756,
 363,
 109,
 477,
 208,
 228,
 84,
 1910,
 72,
 618,
 125,
 5046,
 194,
 154,
 447,
 852,
 542,
 4424,
 102,
 343,
 2883,
 245,
 583,
 839,
 36,
 576,
 718,
 3208,
 607,
 108,
 231,
 1318,
 780,
 4433,
 1720,
 210,
 4121,
 327,
 1689,
 2132,
 192,
 1392,
 457,
 2295,
 183,
 3762,
 562,
 138,
 264,
 2373,
 2996,
 1041,
 1929,
 1362,
 3997,
 2366,
 1637,
 1005,
 55,
 1451,
 170,
 85,
 52,
 136,
 211,
 149,
 1321,
 2304,
 268,
 761,
 10,
 399,
 2551,
 92,
 49,
 3166,
 480,
 4562,
 176,
 

In [255]:
# inverse_indices_old = [None] * len(words_to_shuffle)
# used = [False] * len(words_to_shuffle)

# for i, word in enumerate(shuffled_words):
#     for j, (orig_word, flag) in enumerate(zip(words_to_shuffle, used)):
#         if not flag and word == orig_word:
#             inverse_indices[i] = shuffle_indices[j]
#             used[j] = True
#             break

In [256]:
# len(inverse_indices_old)

In [257]:
# # Choose one of the following (set only one to True)
# shuffle_paragraphs_flag = True
# shuffle_sentences_flag = False
# shuffle_words_flag = True

# if shuffle_words_flag:
#     LLM_input_sequence, inverse_indices = shuffle_words(LLM_input_sequence)
# else:
#     inverse_indices = list(range(len(LLM_input_sequence)))

## 2. compute LLM representations for these words.

In [647]:
LLM_input_sequence=shuffled_sequence.copy()

In [648]:
# Set to a writable location, like /tmp or your home dir
HF_home = "/tmp/huggingface_cache"  # or "/home/youruser/.cache/huggingface"

os.makedirs(HF_home, exist_ok=True)

In [649]:
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, cache_dir=HF_home)
model = AutoModel.from_pretrained(model_name, cache_dir=HF_home).to("cpu")
print("Loaded model and tokenizer.")

Loaded model and tokenizer.


In [650]:
strings_to_find = LLM_input_sequence # we search for these words in sequential order in the full prompt to obtain avg representations for these.
prompt_text = " ".join(LLM_input_sequence)

In [651]:
prompt_text[:500], len(prompt_text)

('Blood was pounding in his ears. It\'s really very selfish of you." \n\n "And it\'s really none of your business," said Harry. \n\n "Madam Hooch told us not to move -- you\'ll get us all into trouble." \n\n Harry ignored her. The Fat Lady had gone on a nighttime visit and Hermione was locked out of Gryffindor Tower. "Don\'t you care about Gryffindor, do you only care about yourselves, I don\'t want Slytherin to win the House Cup, and you\'ll lose all the points I got from Professor McGonagall for knowing abo',
 28670)

In [652]:
# compute representations 
occurrences, representations = get_ordered_representations(
    ordered_strings_to_find=strings_to_find,
    prompt=prompt_text,
    tokenizer=tokenizer,
    model=model
)
print("Computed word level representations for the input sequence.")

Computed word level representations for the input sequence.


In [653]:
representations[0].shape

torch.Size([5176, 2048])

## 3. Map from word-level LLM representations to TR level representations via Lanczos resampling

In [654]:
layer_idx = -7

In [655]:
# Step: Reorder the model output (shuffled order) back to original order using inverse_indices
reordered_representations = representations[layer_idx][inverse_indices]

In [656]:

# word_times must be ordered the same as reordered_representations
reordered_word_times = word_times[inverse_indices]  # Make sure this works after fixing inverse_indices

# Ensure representation and word_times match
assert reordered_representations.shape[0] == reordered_word_times.shape[0], "Mismatch in # of tokens and timings"


In [657]:
reordered_representations


tensor([[-0.1033, -0.0761, -0.0459,  ...,  0.0026,  0.1429,  0.0486],
        [-0.1258, -0.1048, -0.2286,  ..., -0.0829, -0.0212, -0.1202],
        [-0.0358, -0.0190, -0.0780,  ..., -0.0881, -0.1679,  0.1194],
        ...,
        [-0.0146, -0.0490,  0.1114,  ..., -0.2213,  0.0413,  0.0026],
        [-0.0519, -0.0470,  0.2518,  ...,  0.0563, -0.0777, -0.0073],
        [-0.0614, -0.0540,  0.1222,  ..., -0.0139, -0.1035, -0.0366]])

In [658]:
reordered_representations[0].shape

torch.Size([2048])

In [659]:
interpolated_representations = lanczosinterp2D(
    reordered_representations.to("cpu"),             # shape: (n_samples_input, n_dim)
    word_times,      # shape: (n_samples_input,)
    fmri_time,     # shape: (n_samples_target,)
    window=3,         # (optional) number of lobes for the window
    cutoff_mult=1.0,  # (optional) cutoff frequency multiplier
    rectify=False     # (optional) rectification
)
print("Interpolated LLM representations to match fMRI TRs via Lanczos downsampling.")

Doing lanczos interpolation with cutoff=0.500 and 3 lobes.
Interpolated LLM representations to match fMRI TRs via Lanczos downsampling.


In [660]:
interpolated_representations.shape

(1351, 2048)

## 4. Concatenate last x TRs of the LLM representation
No need to filter the first x features because we skip those anyway in the next step.

In [296]:
N_lag = 4  # number of previous TRs to concatenate
interpolated_representations = concat_past_features(torch.from_numpy(interpolated_representations), N_lag).numpy()

In [297]:
interpolated_representations.shape

(1351, 10240)

## 5. Filter out TRs at the boundary of multiple runs

In [298]:
# Delete the first 2 and last 1 elements from each experiment run
n_begin = 20
n_last = 15
mask = fmri_runs # This is an integer mask indicating which TRs belong to which run.
final_representations = delete_block_edges(interpolated_representations, 
                                           mask, 
                                           n_begin, 
                                           n_last, 
                                           axis=0) # axis 0: time-dimension is the first in our data
print(f"Deleted edges from the LLM representations to match the filtering of fMRI runs (removed first {n_begin} and last {n_last} of every run).")

Deleted edges from the LLM representations to match the filtering of fMRI runs (removed first 20 and last 15 of every run).


In [299]:
fmri_data.shape

(1211, 25263)

In [300]:
final_representations.shape

(1211, 10240)

## 6. Run the nested blocked cross-validation to find the best alpha per voxel for ridge regression.

In [301]:
# Convert data to torch tensors and move to GPU
X = torch.tensor(final_representations, dtype=torch.float32, device="cpu") # shape (n_TRs, n_features)
y = torch.tensor(fmri_data, dtype=torch.float32, device="cpu") # shape (n_TRs, n_voxels)

# Set parameters for crossvalidation
split_function = "blocked" # divide data into uniform folds
block_labels = None  # Not needed for blocked splitting - useful for experiment-wise CV folds
n_splits_outer = 4   # Four blocks for outer CV (must be larger than 1)
n_splits_inner = 3   # Three blocks for inner CV (must be larger than 1)
gap = 15 # number of TRs to skip/discard in between train and test to avoid leakage
alphas = [0.1,1,10,100,1000,10000] # default vals from https://github.com/mtoneva/brain_language_nlp/blob/master/utils/utils.py 

In [302]:
X.shape, y.shape

(torch.Size([1211, 10240]), torch.Size([1211, 25263]))

In [303]:
device = "cpu"
print("Starting nested blocked cross-validation to find the best alpha per voxel for ridge regression")
# Obtain the best ridge regression penalties for each voxel independently (~25k alphas)
best_alphas, outer_scores = nested_blocked_cv(
    X, y,
    split_function=split_function,
    block_labels=block_labels,
    n_splits_outer=n_splits_outer,
    n_splits_inner=n_splits_inner,
    gap=gap,
    alphas=alphas,
    device=device
)
print("Best alphas found:", best_alphas)

Starting nested blocked cross-validation to find the best alpha per voxel for ridge regression


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Ridge Regression models fitted:   3%|▎         | 3/96 [00:29<15:07,  9.76s/it]

torch.Size([894, 25263]) y_train_outer shape
(25263,) best alphas for fold


Ridge Regression models fitted:   6%|▋         | 6/96 [00:57<14:01,  9.35s/it]

torch.Size([894, 25263]) y_train_outer shape
(25263,) best alphas for fold


...

torch.Size([891, 25263]) y_train_outer shape
(25263,) best alphas for fold


Ridge Regression models fitted:  78%|███████▊  | 75/96 [16:23<03:21,  9.60s/it]

torch.Size([891, 25263]) y_train_outer shape
(25263,) best alphas for fold
Best alphas for fold: 25263 0 voxels with no best alpha found
(4, 25263) best alphas shape


Ridge Regression models fitted:  79%|███████▉  | 76/96 [17:47<04:40, 14.04s/it]

Outer scores for this fold: [ 0.09220495  0.05191445  0.11446941 ... -0.06565859  0.01026042
  0.05795   ]
[array([-0.0332336 , -0.07788308, -0.11560593, ...,  0.00968779,
        0.05811946,  0.01745196], shape=(25263,), dtype=float32), array([ 0.05543184,  0.08387576, -0.00931988, ...,  0.01935829,
        0.05239794,  0.04088382], shape=(25263,), dtype=float32), array([0.09816635, 0.12470783, 0.09702691, ..., 0.02867154, 0.08311213,
       0.10760191], shape=(25263,), dtype=float32), array([ 0.09220495,  0.05191445,  0.11446941, ..., -0.06565859,
        0.01026042,  0.05795   ], shape=(25263,), dtype=float32)] outer scores
Best alphas found: [0.1, 1000.0, 0.1, 0.1, 10000.0, 0.1, 1000.0, 1000.0, 10000.0, 10000.0, 1.0, 1000.0, 0.1, 1000.0, 10000.0, 10000.0, 1000.0, 1000.0, 1.0, 10000.0, 0.1, 0.1, 0.1, 0.1, 1.0, 0.1, 10000.0, 1.0, 1.0, 10.0, 0.1, 1.0, 1000.0, 1000.0, 1.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 1.0, 1.0, 1.0, 10000.0, 10000.0, 10000.0, 10000.0, 

In [ ]:
block_labels = [[0 for _ in range(X.shape[0])]] # everything comes from the same story so we assume identical block_labels.